<a href="https://colab.research.google.com/github/hamsungmin/DataTrainAnalysis/blob/main/project_week14_%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%99%9C%EC%9A%A9%ED%95%9C_AI_%EC%B1%97%EB%B4%87_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. 데이터 전처리 및 청크 생성**

<aside>

**진행 프로세스**

스마트폰 제조공정에서는 각 단계별 품질기준, 검사 절차, 불량 대응 방안 등이 포함된 품질관리 매뉴얼 문서(HTML) 를 운영 중입니다.
이 문서는 향후 품질분석 GPT 챗봇의 핵심 학습 데이터로 활용되며, 이를 위해 HTML 문서를 자동으로 섹션 단위로 **구조화(청크화)** 해야 합니다.

- report.html 파일을 불러와 **<section> 단위로 내용을 분석**하고, 각 섹션을 하나의 **품질매뉴얼 청크(Chunk)를 구성**합니다.  (청크구성은 최종결과물에 따라 품질이 결정됨으로 정답은 없음)
    - 각 섹션의 id와 data-section 속성을 함께 추출하여 고유 식별자로 활용
    - 청크별 텍스트를 [섹션명] 내용 형식으로 구성하여 구조화
    - chunks와 chunk_ids를 딕셔너리 형태로 저장하여 다음 단계에서 활용 가능하도록 구성
- 추출된 청크 데이터를 검증하고, 결과를 chunks_data.pkl 파일로 저장합니다.
    - 전체 청크 개수와 일부 예시를 출력하여 추출 결과 확인
</aside>

In [ ]:
from bs4 import BeautifulSoup
import pickle

with open('/content/report.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, 'html.parser')

chunks = []
chunk_ids = []

for section in soup.find_all(attrs={'data-section': True}):
    section_id = section.get('id')
    section_name = section.get('data-section')
    section_text = section.get_text(strip=True)

    chunk = f"[{section_name}] {section_text}"

    chunks.append(chunk)
    chunk_ids.append(section_id)

chunks_data = {
    'chunks': chunks,
    'chunk_ids': chunk_ids
}

print(f"총 추출된 청크: {len(chunks_data['chunks'])}")
print("추출된 청크 예시")
for i in range(min(5, len(chunks_data['chunks']))):
    print(f"- {chunks_data['chunks'][i][:200]}...") # Print first 200 characters of the chunk

with open('chunks_data.pkl', 'wb') as f:
    pickle.dump(chunks_data, f)

print("청크 데이터 저장 완료 to chunks_data.pkl")

총 추출된 청크: 22
추출된 청크 예시
- [개요] 1. 개요1.1 목적본 매뉴얼은ABC전자 스마트폰 제조공정에서의 품질관리 기준과 절차를 정의하여고객 만족도 향상과불량률 최소화를 목표로 합니다.1.2 적용범위스마트폰 조립라인 전체 공정부품 입고검사부터 완제품 출하까지모든 품질관리 담당자 및 생산직 근로자...
- [목적] 1.1 목적본 매뉴얼은ABC전자 스마트폰 제조공정에서의 품질관리 기준과 절차를 정의하여고객 만족도 향상과불량률 최소화를 목표로 합니다....
- [적용범위] 1.2 적용범위스마트폰 조립라인 전체 공정부품 입고검사부터 완제품 출하까지모든 품질관리 담당자 및 생산직 근로자...
- [주요 공정별 품질 기준] 2. 주요 공정별 품질 기준2.1 부품 입고검사 (IQC)검사항목디스플레이 패널: 화소 불량0.01% 이하배터리: 용량 편차±3% 이내카메라 모듈: 해상도 테스트100% 합격메인보드:전기적 특성 검사 필수불량 처리불량률5% 초과 시 전량 반품공급업체 개선요구서 발행2.2 조립공정 (Assembly)주요 체크포인트디스플레이 부착접착...
- [부품 입고검사 (IQC)] 2.1 부품 입고검사 (IQC)검사항목디스플레이 패널: 화소 불량0.01% 이하배터리: 용량 편차±3% 이내카메라 모듈: 해상도 테스트100% 합격메인보드:전기적 특성 검사 필수불량 처리불량률5% 초과 시 전량 반품공급업체 개선요구서 발행...
청크 데이터 저장 완료 to chunks_data.pkl


**2. 임베딩생성**

<aside>

**진행 프로세스**

스마트폰 제조공정에서 사용되는 품질관리 매뉴얼 청크 데이터(chunks_data.pkl) 는 이미 섹션 단위로 구조화된 텍스트지만, 이 상태로는 GPT 모델이 각 청크의 의미적 유사성을 이해하거나 검색할 수 없습니다.  **따라서, 각 청크를 벡터(Embedding) 형태로 변환해 의미를 수치화해야 합니다.**
이를 통해 문장 간의 유사도를 계산하고, 사용자가 질문했을 때 가장 관련성 높은 청크를 효율적으로 검색할 수 있습니다.

- `chunks_data.pkl` chunks_data.pkl파일을 불러와 각 청크의 텍스트를 **의미 벡터(Embedding)** 로 변환하는 프로그램을 작성합니다.
    - OpenAI API를 이용하여 텍스트를 임베딩 벡터로 변환
    - 적절한 임베딩 모델(`text-embedding-3-small` 등)을 선택하여 벡터 생성
    - 각 청크의 ID와 임베딩 결과를 매핑하여 구조적으로 관리
- 생성된 임베딩 벡터를 검증하고, 결과를 `embeddings_data.pkl` 파일로 저장합니다.
    - 전체 청크 개수, 벡터 차원, 배열 크기 등을 출력하여 정상 변환 여부 확인
    - 일부 샘플 벡터(예: 첫 번째 청크의 앞 10개 값)를 출력해 임베딩 결과를 검증
    - `embeddings`, `chunks`, `chunk_ids`를 포함하는 딕셔너리 형태로 저장
</aside>

In [ ]:
import pickle
import os
from openai import OpenAI
import numpy as np
from google.colab import userdata

with open('chunks_data.pkl', 'rb') as f:
    chunks_data = pickle.load(f)

chunks = chunks_data['chunks']
chunk_ids = chunks_data['chunk_ids']

openai_api_key = userdata.get('OPENAI_API_KEY')
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY not found in Colab Secrets. Please add it.")

client = OpenAI(api_key=openai_api_key)

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

embeddings = []
total_chunks = len(chunks)
print("임베딩 생성 중...")
for i, chunk in enumerate(chunks):
    embedding = get_embedding(chunk)
    embeddings.append(embedding)
    print(f"처리 중 : {i+1}/{total_chunks}")

embeddings_np = np.array(embeddings)

embeddings_data = {
    'embeddings': embeddings_np,
    'chunks': chunks,
    'chunk_ids': chunk_ids
}

print(f"청크 개수: {len(embeddings_data['embeddings'])}")
print(f"벡터 차원: {embeddings_data['embeddings'].shape[1]}")
print(f"배열 크기: {embeddings_data['embeddings'].shape}")
print("벡터 샘플 처음 10개:")
print(embeddings_data['embeddings'][0][:10])

with open('embeddings_data.pkl', 'wb') as f:
    pickle.dump(embeddings_data, f)

print("임베딩 데이터 저장 완료 : embeddings_data.pkl")

임베딩 생성 중...
처리 중 : 1/22
처리 중 : 2/22
처리 중 : 3/22
처리 중 : 4/22
처리 중 : 5/22
처리 중 : 6/22
처리 중 : 7/22
처리 중 : 8/22
처리 중 : 9/22
처리 중 : 10/22
처리 중 : 11/22
처리 중 : 12/22
처리 중 : 13/22
처리 중 : 14/22
처리 중 : 15/22
처리 중 : 16/22
처리 중 : 17/22
처리 중 : 18/22
처리 중 : 19/22
처리 중 : 20/22
처리 중 : 21/22
처리 중 : 22/22
청크 개수: 22
벡터 차원: 1536
배열 크기: (22, 1536)
벡터 샘플 처음 10개:
[ 0.00648111  0.06691471 -0.01022252  0.00642101 -0.00710218 -0.01945335
  0.02155695 -0.00035217 -0.01017244 -0.00572982]
임베딩 데이터 저장 완료 : embeddings_data.pkl


**3.  FAISS 인덱스 생성**

<aside>

**진행 프로세스**

생성된 청크 데이터는 이미 텍스트 임베딩 과정을 통해 의미 벡터로 변환되어 있지만,  수천 개 이상의 벡터를 직접 비교하여 검색하는 것은 비효율적이므로, 벡터데이터베이스인 **FAISS**(Facebook AI Similarity Search) 를 이용해 **빠르고 정확한 유사도 검색 인덱**스를 **구축**해야 합니다.

- FAISS 라이브러리를 이용해 인덱스를 생성합니다. (인덱스 종류는 선택함)
    - 대량의 임베딩 데이터를 **고속 검색 가능한 벡터 인덱스**로 변환
    - L2 거리 기반으로 **의미적 유사 청크를 탐색**할 수 있는 구조 설계
    - 인덱스의 검색 결과를 검증하고, 향후 **RAG 질의응답 시스템의 핵심 검색 모듈**로 활용
- **인덱스, 임베딩, 청크, 청크 ID**를 포함한 객체를 저장합니다.
    - `rag_system.pkl` 파일로 저장
</aside>

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 47.7 MB/s eta 0:00:00


In [ ]:
import pickle
import numpy as np
import faiss
import os

with open('embeddings_data.pkl', 'rb') as f:
    embeddings_data = pickle.load(f)

embeddings = embeddings_data['embeddings']
chunks = embeddings_data['chunks']
chunk_ids = embeddings_data['chunk_ids']

embeddings = embeddings.astype('float32')

dimension = embeddings.shape[1]

print(f"데이터 로드 완료.")
print(f"벡터 개수: {len(embeddings)}")
print(f"벡터 차원: {dimension}")


index = faiss.IndexFlatL2(dimension)

index.add(embeddings)

print(f"FAISS 인덱스 생성 완료!")
print(f"인덱스 타입: {type(index)}")
print(f"저장된 백터 개수 : {index.ntotal} ")


rag_system_data = {
    'index': index,
    'embeddings': embeddings,
    'chunks': chunks,
    'chunk_ids': chunk_ids
}

with open('rag_system.pkl', 'wb') as f:
    pickle.dump(rag_system_data, f)

print("RAG 시스템 저장 완료 : rag_system.pkl")


print("\n" + "="*50)
print("첫 번째 청크와 가장 유사한 상위 3개 청크 검색 결과")
print("="*50)

query_embedding = embeddings[0].reshape(1, -1).astype('float32')

k = 4
distances, indices = index.search(query_embedding, k)

print(f"쿼리 청크 ID: {chunk_ids[0]}")
print(f"쿼리 청크 (처음 200자): {chunks[0][:200]}...")
print("-" * 50)
print("유사도 검색 결과 (상위 3개):")

for i in range(1, k):
    similar_chunk_index = indices[0][i]
    similar_chunk_distance = distances[0][i]
    similar_chunk = chunks[similar_chunk_index]
    similar_chunk_id = chunk_ids[similar_chunk_index]

    print(f"{i}위:")
    print(f"  청크 ID: {similar_chunk_id}")
    print(f"  청크 (처음 200자): {similar_chunk[:200]}...")
    print(f"  유사도 거리 (L2): {similar_chunk_distance}")
    print("-" * 20)



데이터 로드 완료.
벡터 개수: 22
벡터 차원: 1536
FAISS 인덱스 생성 완료!
인덱스 타입: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
저장된 백터 개수 : 22 
RAG 시스템 저장 완료 : rag_system.pkl

첫 번째 청크와 가장 유사한 상위 3개 청크 검색 결과
쿼리 청크 ID: sec-1
쿼리 청크 (처음 200자): [개요] 1. 개요1.1 목적본 매뉴얼은ABC전자 스마트폰 제조공정에서의 품질관리 기준과 절차를 정의하여고객 만족도 향상과불량률 최소화를 목표로 합니다.1.2 적용범위스마트폰 조립라인 전체 공정부품 입고검사부터 완제품 출하까지모든 품질관리 담당자 및 생산직 근로자...
--------------------------------------------------
유사도 검색 결과 (상위 3개):
1위:
  청크 ID: sec-1-1
  청크 (처음 200자): [목적] 1.1 목적본 매뉴얼은ABC전자 스마트폰 제조공정에서의 품질관리 기준과 절차를 정의하여고객 만족도 향상과불량률 최소화를 목표로 합니다....
  유사도 거리 (L2): 0.2851194441318512
--------------------
2위:
  청크 ID: sec-1-2
  청크 (처음 200자): [적용범위] 1.2 적용범위스마트폰 조립라인 전체 공정부품 입고검사부터 완제품 출하까지모든 품질관리 담당자 및 생산직 근로자...
  유사도 거리 (L2): 0.7820435762405396
--------------------
3위:
  청크 ID: sec-2
  청크 (처음 200자): [주요 공정별 품질 기준] 2. 주요 공정별 품질 기준2.1 부품 입고검사 (IQC)검사항목디스플레이 패널: 화소 불량0.01% 이하배터리: 용량 편차±3% 이내카메라 모듈: 해상도 테스트100% 합격메인보드:전기적 특성 검사 필수불량 처리불량률5% 초과 시 전량 반품공급업체 개선요구서 발행2.2 조립공정 (Assembly)주요 체크포인트

**4 .질문검색 및 답변생성**

<aside>

**진행 프로세스**

축적된 **품질관리 매뉴얼 데이터**를 효율적으로 활용하기 위해서는, 사용자가 질의한 내용을 문서 내에서 **의미적으로 검색하고, GPT를 통해 정확한 답변을 생성**할 수 있어야 합니다.

이단계는 RAG 시스템의 **최종 단계로서**, 앞서 구축된 **FAISS 인덱스와 임베딩 데이터**를 이용하여 다음을 달성하는 것을 목표로 합니다:

- **사용자의 질문을 임베딩 벡터로 변환하여** 단순 키워드 검색이 아닌, **의미 기반 검색(semantic retrieval)** 을 진행합니다.
    - 사용자의 자연어 질문을 OpenAI 임베딩 모델을 통해 벡터로 변환
    - 생성된 질문 벡터를 활용해 의미적으로 유사한 문서를 검색할 수 있도록 준비
- **FAISS 인덱스를 활용해 관련 문서를 탐색하고**  **품질관리 GPT 챗봇의 실질적 질의응답 기능을 완성하여 GPT로 답변을 생성합니다.**
    - 변환된 질문 벡터를 FAISS 인덱스에 입력하여 **가장 유사한 문서 청크**를 효율적으로 탐색
    - 검색된 문서들을 GPT의 **컨텍스트(context)** 로 제공하여, 품질관리 전문가 수준의 답변을 자동 생성
</aside>

In [ ]:
import pickle
import numpy as np
import faiss
from openai import OpenAI
from google.colab import userdata
import textwrap

# 저장된 RAG 시스템 데이터 불러오기
try:
    with open('rag_system.pkl', 'rb') as f:
        rag_system_data = pickle.load(f)
    index = rag_system_data['index']
    embeddings = rag_system_data['embeddings']
    chunks = rag_system_data['chunks']
    chunk_ids = rag_system_data['chunk_ids']
    print("RAG 시스템 데이터 불러오기 완료.")
except FileNotFoundError:
    print("rag_system.pkl 파일을 찾을 수 없습니다. 이전 단계를 먼저 완료해주세요.")
    exit() # 파일이 없으면 스크립트 종료

# OpenAI 클라이언트 초기화
# Colab Secrets에 'OPENAI_API_KEY'로 API 키를 설정해주세요.
openai_api_key = userdata.get('OPENAI_API_KEY')
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY가 Colab Secrets에 설정되지 않았습니다. 추가해주세요.")

client = OpenAI(api_key=openai_api_key)

# 임베딩 생성 함수 (이전 단계에서 사용한 함수 재사용)
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   # Fix: Access the embedding list from the data attribute and convert to numpy array before reshaping
   return np.array(client.embeddings.create(input = [text], model=model).data[0].embedding)

# 질의응답 함수 정의
def ask_rag(query, index, chunks, chunk_ids, client, k=3):
    # 사용자의 질문 임베딩 생성
    query_embedding = get_embedding(query).reshape(1, -1).astype('float32')

    # FAISS 인덱스에서 유사한 청크 검색
    # k+1을 사용하는 이유는 검색 결과의 첫 번째는 항상 자기 자신(질문 임베딩)일 가능성이 높기 때문입니다.
    distances, indices = index.search(query_embedding, k + 1)

    # 검색된 청크 추출 (자기 자신 제외)
    retrieved_chunks = [chunks[i] for i in indices[0] if chunk_ids[indices[0][0]] != chunk_ids[i]][:k]
    retrieved_chunk_ids = [chunk_ids[i] for i in indices[0] if chunk_ids[indices[0][0]] != chunk_ids[i]][:k]


    print("\n" + "="*50)
    print(f"'{query}' 질문에 대한 검색 결과 (상위 {k}개 청크):")
    print("="*50)
    for i in range(len(retrieved_chunks)):
        print(f"{i+1}위 청크 ID: {retrieved_chunk_ids[i]}")
        print(f"내용 (처음 200자): {retrieved_chunks[i][:200]}...")
        print("-" * 20)


    # GPT 모델에 전달할 프롬프트 구성
    # 검색된 청크를 컨텍스트로 제공합니다.
    context = "\n\n".join(retrieved_chunks)
    prompt = f"다음 정보를 참고하여 질문에 답변하세요:\n\n{context}\n\n질문: {query}\n\n답변:"

    # GPT 모델을 사용하여 답변 생성
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo", # 또는 다른 적절한 모델 선택
            messages=[
                {"role": "system", "content": "당신은 품질관리 매뉴얼에 대한 질문에 답변하는 유용한 챗봇입니다."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500, # 답변 최대 길이 설정
            temperature=0.7 # 답변의 다양성 조절
        )
        answer = response.choices[0].message.content
        print("\n" + "="*50)
        print("GPT 모델의 답변:")
        print("="*50)
        # 답변 줄바꿈 처리
        print(textwrap.fill(answer, width=80))
    except Exception as e:
        print(f"\nGPT 모델 답변 생성 중 오류 발생: {e}")
        answer = "답변을 생성하지 못했습니다."

    return answer

# 테스트 질문
user_query = "부품 입고 검사 기준이 뭐야?"

# 질의응답 실행
rag_response = ask_rag(user_query, index, chunks, chunk_ids, client, k=3)

# 다른 질문으로 테스트 가능
# user_query = "최근 고객 불만 사항은 뭐야?"
# rag_response = ask_rag(user_query, index, chunks, chunk_ids, client, k=3)

RAG 시스템 데이터 불러오기 완료.

'부품 입고 검사 기준이 뭐야?' 질문에 대한 검색 결과 (상위 3개 청크):
1위 청크 ID: sec-2
내용 (처음 200자): [주요 공정별 품질 기준] 2. 주요 공정별 품질 기준2.1 부품 입고검사 (IQC)검사항목디스플레이 패널: 화소 불량0.01% 이하배터리: 용량 편차±3% 이내카메라 모듈: 해상도 테스트100% 합격메인보드:전기적 특성 검사 필수불량 처리불량률5% 초과 시 전량 반품공급업체 개선요구서 발행2.2 조립공정 (Assembly)주요 체크포인트디스플레이 부착접착...
--------------------
2위 청크 ID: sec-3-2
내용 (처음 200자): [불량 대응 절차] 3.2 불량 대응 절차즉시 조치사항불량품 격리 및 태그 부착불량 원인 분석 시작동일 로트 전수검사 실시근본원인 분석5Why 분석법 적용생산조건 재검토공급업체 품질 점검...
--------------------
3위 청크 ID: sec-7
내용 (처음 200자): [교육 및 훈련] 7. 교육 및 훈련7.1 품질교육 프로그램신입사원 교육 (40시간)품질 기본개념불량 식별 방법측정장비 사용법고객 중심 사고기존 직원 보수교육 (분기별 8시간)신규 품질기준 안내불량 사례 분석개선제안 활동7.2 자격증 취득 지원품질관리기사6시그마 벨트ISO 9001 심사원...
--------------------

GPT 모델의 답변:
부품 입고 검사 기준은 다음과 같습니다: - 디스플레이 패널: 화소 불량 0.01% 이하 - 배터리: 용량 편차 ±3% 이내 - 카메라 모듈:
해상도 테스트 100% 합격 - 메인보드: 전기적 특성 검사 필수 - 불량 처리: 불량률 5% 초과 시 전량 반품 - 공급업체 개선요구서 발행
